# Final Rate Card Assessment

In [1]:
import pandas as pd

In [2]:
bc_df = pd.read_excel("./NYJ-Data/Broadcast_Exposures.xlsx") # broadcast exposures
vb_df = pd.read_excel("./NYJ-Data/Videoboard_Features.xlsx") # videoboard features
sm_df = pd.read_csv("./NYJ-Data/AllContentSeriesPosts.csv") # social media posts

# convert Instragram Stories Tap Count to Engagement Rate
for i, row in sm_df.iterrows():
    if row['ServiceType'] == 'Instagram Story':
        new_val = row['TapBackwardCount']/row['TapForwardCount']
        sm_df.at[i, 'EngagementRate'] = new_val

sm_df.dropna(subset= ['EngagementRate'], inplace=True)



bc_df = bc_df[['Asset', 'Audience', 'Media_Value_USD', 'Seen_Hits_Seconds']]
vb_df = vb_df[['Asset', 'Video_Total_Seconds', 'Logo_Average_Clarity', 'Logo_Average_Size', 'Logo_Total_Seconds']]
sm_df = sm_df[['ServiceType','Content_Series_Name', 'EngagementRate', 'PostValue']]

In [3]:
# Min-Max Normalization

df = bc_df.drop('Asset', axis=1)
df_norm = (df - df.min()) / (df.max() - df.min())
bc_df = pd.concat((bc_df['Asset'], df_norm), axis=1)

df = vb_df.drop('Asset', axis=1)
df_norm = (df - df.min()) / (df.max() - df.min())
vb_df = pd.concat((vb_df['Asset'], df_norm), axis=1)

df = sm_df.drop(['Content_Series_Name', 'ServiceType'], axis=1)
df_norm = (df - df.min()) / (df.max() - df.min())
sm_df = pd.concat((sm_df[['Content_Series_Name', 'ServiceType']], df_norm), axis=1)

In [4]:
# Constants

T_VAL = 1.96 # t-value for 95% confidence interval
OUTLIER = 50 # threshold for outlier removal determined by looking at distribution of videoboard, broadcast, and social media counts

## VideoBoard Scoring

In [5]:
def VB_Scoring(sec_weight, clarity_weight, size_weight):
    """
        Calculate the score for videoboard assets
        Input:  weights for videoboard variables: Video_Total_Seconds, Logo_Average_Clarity, Logo_Average_Size
        Output: Score for videoboard assets
    """
    vb_vars = vb_df.groupby('Asset').agg(['count', 'mean', 'std']).reset_index()

    # filter out series that don't meet median threshold
    median_count = vb_vars['Video_Total_Seconds'].where(vb_vars['Video_Total_Seconds']['count'] < OUTLIER)['count'].median()

    vid_sec = vb_vars[['Asset', 'Video_Total_Seconds']].where(vb_vars['Video_Total_Seconds']['count'] >= median_count).dropna()
    logo_sec = vb_vars[['Asset', 'Logo_Total_Seconds']].where(vb_vars['Logo_Total_Seconds']['count'] >= median_count).dropna()
    clarity = vb_vars[['Asset', 'Logo_Average_Clarity']].where(vb_vars['Logo_Average_Clarity']['count'] >= median_count).dropna()
    size = vb_vars[['Asset', 'Logo_Average_Size']].where(vb_vars['Logo_Average_Size']['count'] >= median_count).dropna()

    # rename columns
    vid_sec.columns = vid_sec.columns.map(' '.join)
    logo_sec.columns = logo_sec.columns.map(' '.join)
    clarity.columns = clarity.columns.map(' '.join)
    size.columns = size.columns.map(' '.join)

    # recombine into one dataframe
    recombined = pd.merge(logo_sec, clarity, on='Asset ')
    recombined = pd.merge(recombined, size, on='Asset ')
    addSeries = vb_vars[vb_vars.Asset.str.contains('Did You Know?')]
    addSeries.columns = addSeries.columns.map(' '.join)
    recombined = recombined.append(addSeries) ## add this series to connect social media

    Scores = pd.DataFrame(columns=['Asset', 'Low', 'Average', 'High'])

    # calculate 95% confidence interval scores
    for index,row in recombined.iterrows():
        low = sec_weight * (row['Logo_Total_Seconds mean'] - T_VAL*row['Logo_Total_Seconds std']) + clarity_weight * (row['Logo_Average_Clarity mean'] - T_VAL*row['Logo_Average_Clarity std'] )+ size_weight * (row['Logo_Average_Size mean'] - T_VAL*row['Logo_Average_Size std'])
        average = sec_weight* row['Logo_Total_Seconds mean'] + clarity_weight* row['Logo_Average_Clarity mean'] + size_weight * row['Logo_Average_Size mean']
        high = sec_weight * (row['Logo_Total_Seconds mean'] + T_VAL*row['Logo_Total_Seconds std']) + clarity_weight * (row['Logo_Average_Clarity mean'] + T_VAL*row['Logo_Average_Clarity std'] )+ size_weight * (row['Logo_Average_Size mean'] + T_VAL*row['Logo_Average_Size std'])

        Scores.loc[index] = [row['Asset '], low, average, high]
    
    # adjust all scores to get rid of negative values
    min_score = Scores['Low'].min()
    Scores['Low'] = Scores['Low'] + abs(min_score) + abs(Scores['Low'].mean())
    Scores['Average'] = Scores['Average'] + abs(min_score) + abs(Scores['Low'].mean())
    Scores['High'] = Scores['High'] + abs(min_score) + abs(Scores['Low'].mean())

    # sort scores for debugging purposes
    Scores = Scores.sort_values(by='Average', ascending=False)

    return Scores

## Broadcast Scoring

In [6]:
def BC_Scoring(aud_weight=1.5, val_weight=2, hit_weight=.75):
    """
        Calculate the score for broadcast assets
        Input:  weights for broadcast variables: Audience, Media_Value_USD, Seen_Hits_Seconds
        Output: Score for broadcast assets

    """
    bc_vars = bc_df.groupby('Asset').agg(['count', 'mean', 'std']).reset_index()

    # filter out series that don't meet median threshold
    median_count = bc_vars['Audience'].where(bc_vars['Audience']['count'] < OUTLIER)['count'].median()

    audience = bc_vars[['Asset', 'Audience']].where(bc_vars['Audience']['count'] >= median_count).dropna()
    value = bc_vars[['Asset', 'Media_Value_USD']].where(bc_vars['Media_Value_USD']['count'] >= median_count).dropna()
    hits = bc_vars[['Asset', 'Seen_Hits_Seconds']].where(bc_vars['Seen_Hits_Seconds']['count'] >= median_count).dropna()

    # rename columns
    audience.columns = audience.columns.map(' '.join)
    value.columns = value.columns.map(' '.join)
    hits.columns = hits.columns.map(' '.join)

    # recombine into one dataframe
    recombined = pd.merge(audience, value, on='Asset ')
    recombined = pd.merge(recombined, hits, on='Asset ') 

    Scores = pd.DataFrame(columns=['Asset', 'Low', 'Average', 'High'])

    # calculate 95% confidence interval scores
    for index, row in recombined.iterrows():
        low = aud_weight * (row['Audience mean'] - T_VAL * row['Audience std']) + val_weight * (row['Media_Value_USD mean'] - T_VAL * row['Media_Value_USD std'])+ hit_weight * (row['Seen_Hits_Seconds mean'] - T_VAL * row['Seen_Hits_Seconds std'])
        average = aud_weight *row['Audience mean'] + val_weight * row['Media_Value_USD mean'] + hit_weight * row['Seen_Hits_Seconds mean']
        high = aud_weight * (row['Audience mean'] + T_VAL * row['Audience std']) + val_weight * (row['Media_Value_USD mean'] + T_VAL * row['Media_Value_USD std']) + hit_weight * (row['Seen_Hits_Seconds mean'] + T_VAL * row['Seen_Hits_Seconds std'])
        
        Scores.loc[index] = [row['Asset '], low, average, high]
    
    # adjust all scores to get rid of negative values
    min_score = Scores['Low'].min()
    Scores['Low'] = Scores['Low'] + abs(min_score) + abs(Scores['Low'].mean())
    Scores['Average'] = Scores['Average'] + abs(min_score) + abs(Scores['Low'].mean())
    Scores['High'] = Scores['High'] + abs(min_score) + abs(Scores['Low'].mean())

    # sort scores for debugging purposes
    Scores = Scores.sort_values(by='Average', ascending=False)
    
    return Scores


## Social Media Scoring

In [7]:
def SM_Scoring(val_weight, eng_weight):
    """
        Calculate the score for social media assets
        Input:  weights for social media variables: Media_Value_USD, Engagement
        Output: Score for social media assets
    """
    sm_vars = sm_df.groupby('Content_Series_Name').agg(['count', 'mean', 'std']).reset_index()

    # filter out series that don't meet median threshold
    median_count = sm_vars['EngagementRate'].where(sm_vars['EngagementRate']['count'] < OUTLIER)['count'].median()

    eng_rate = sm_vars[['Content_Series_Name', 'EngagementRate']].where(sm_vars['EngagementRate']['count'] >= median_count).dropna()
    post_value = sm_vars[['Content_Series_Name', 'PostValue']].where(sm_vars['PostValue']['count'] >= median_count).dropna()

    # rename columns
    eng_rate.columns = eng_rate.columns.map(' '.join)
    post_value.columns = post_value.columns.map(' '.join)

    # recombine into one dataframe
    recombined = pd.merge(eng_rate, post_value, on='Content_Series_Name ')
    addSeries = sm_vars[sm_vars['Content_Series_Name'].str.contains('Did You Know?')]
    addSeries.columns = addSeries.columns.map(' '.join)
    recombined =recombined.append(addSeries, ignore_index = True) ## add this series to connect videoboard

    Scores = pd.DataFrame(columns=['Asset', 'Low', 'Average', 'High'])

    # calculate 95% confidence interval scores
    for index, row in recombined.iterrows():
        low =  eng_weight * (row['EngagementRate mean'] - T_VAL * row['EngagementRate std']) + val_weight * (row['PostValue mean'] - T_VAL * row['PostValue std'])
        average = eng_weight * (row['EngagementRate mean']) + val_weight * (row['PostValue mean'])
        high = eng_weight * (row['EngagementRate mean'] + T_VAL * row['EngagementRate std']) + val_weight * (row['PostValue mean'] + T_VAL * row['PostValue std'])

        Scores.loc[index] = [row['Content_Series_Name '], low, average, high]
    
    # adjust all scores to get rid of negative values
    min_score = Scores['Low'].min()
    Scores['Low'] = Scores['Low'] + abs(min_score) + abs(Scores['Low'].mean())
    Scores['Average'] = Scores['Average'] + abs(min_score) + abs(Scores['Low'].mean())
    Scores['High'] = Scores['High'] + abs(min_score) + abs(Scores['Low'].mean())

    # sort scores for debugging purposes
    Scores = Scores.sort_values(by='Average', ascending=False)

    return Scores

## Pricing Functions

In [8]:
def BC_Pricing(Scores, initPrice):
    """
        Calculate the price for broadcast assets
        Input:  scores for broadcast assets, price for broadcast assets
        Output: price dataframe for broadcast assets
    """
    # Prices
    Prices = pd.DataFrame(columns=['Asset', 'Bronze Tier', 'Silver Tier', 'Gold Tier'])

    # static signage was the most closely related product to our market research
    initScore = Scores[Scores['Asset'] == 'Static Signage']['Average'].values[0]

    # calculate price for each tier, format it to round to the nearest dollar
    for index, row in Scores.iterrows():
        low = "{:,}".format(int((row['Low']/initScore) * initPrice))
        average = "{:,}".format(int((row['Average']/initScore) * initPrice))
        high = "{:,}".format(int((row['High']/initScore) * initPrice))
        
        Prices.loc[index] = [row['Asset'], low, average, high]

    return Prices, int(Prices[Prices['Asset'] == 'Static Signage']['Silver Tier'].values[0].replace(',',''))

In [9]:
def VB_Pricing(Scores, initPrice):
    """
        Calculate the price for video board assets
        Input:  scores for video board assets, price for video board assets
        Output: price dataframe for video board assets
    """
    # Prices
    Prices = pd.DataFrame(columns=['Asset', 'Bronze Tier', 'Silver Tier', 'Gold Tier'])

    initScore = Scores['Average'].mean() # average of the average video board scores

    # calculate price for each tier, format it to round to the nearest dollar
    for index, row in Scores.iterrows():
        low = "{:,}".format(int((row['Low']/initScore) * initPrice))
        average = "{:,}".format(int((row['Average']/initScore) * initPrice))
        high = "{:,}".format(int((row['High']/initScore) * initPrice))
        
        Prices.loc[index] = [row['Asset'], low, average, high]

    return Prices, int(Prices[Prices['Asset'] == 'Did You Know']['Silver Tier'].values[0].replace(',',''))

In [10]:
def SM_Pricing(Scores, initPrice):
    """
        Calculate the price for social media assets
        Input:  scores for social media assets, price for social media assets
        Output: price dataframe for social media assets
    """
    # Prices
    Prices = pd.DataFrame(columns=['Asset', 'Bronze Tier', 'Silver Tier', 'Gold Tier'])
    
    initScore = Scores[Scores['Asset']== 'Did You Know']['Average'].values[0] # Did You Know connects social media to video board

    # calculate price for each tier, format it to round to the nearest dollar
    for index, row in Scores.iterrows():
        low = "{:,}".format(int((row['Low']/initScore) * initPrice))
        average = "{:,}".format(int((row['Average']/initScore) * initPrice))
        high = "{:,}".format(int((row['High']/initScore) * initPrice))
        
        Prices.loc[index] = [row['Asset'], low, average, high]

    return Prices

In [11]:
def overall_Pricing(bc_prices, vb_prices, sm_prices):
    """
        Calculate the overall price for each asset
        Input:  prices for broadcast assets, prices for video board assets, prices for social media assets
        Output: price dataframe for each asset category
    """
    # Prices
    
    bc = bc_prices.copy()
    bc.drop(['Asset'], axis=1, inplace=True)
    bc = bc.apply(lambda x: x.str.replace(',','').astype(int), axis=0)

    vb = vb_prices.copy()
    vb.drop(['Asset'], axis=1, inplace=True)
    vb = vb.apply(lambda x: x.str.replace(',','').astype(int), axis=0)
    
    sm = sm_prices.copy()
    sm.drop(['Asset'], axis=1, inplace=True)
    sm = sm.apply(lambda x: x.str.replace(',','').astype(int), axis=0)

    Prices = pd.DataFrame(columns=['Asset Category', 'Bronze Tier', 'Silver Tier', 'Gold Tier'], 
                          data=[['Broadcast', "{:,}".format(int(bc['Bronze Tier'].mean())), "{:,}".format(int(bc['Silver Tier'].mean())), "{:,}".format(int(bc['Gold Tier'].mean()))], 
                                ['Video Board', "{:,}".format(int(vb['Bronze Tier'].mean())), "{:,}".format(int(vb['Silver Tier'].mean())),  "{:,}".format(int(vb['Gold Tier'].mean()))],
                                ['Social Media',  "{:,}".format(int(sm['Bronze Tier'].mean())),  "{:,}".format(int(sm['Silver Tier'].mean())),  "{:,}".format(int(sm['Gold Tier'].mean()))]])
    return Prices

## Jets ranked 8th most profitable on all sites
So an assumption was made to make the static signage cost 525,000 based on a report saying  
the average cost of advertising is a little under the above amount.
The report stated the range was 250-500,000 and that it would increase by 20% in the next year.
We increased that range by 20% to be 300-600,000 and got the 75th percentile of that to arrive 
at 525,000 .

- https://www.sportskeeda.com/nfl/most-profitable-nfl-franchises-2021#
- https://www.cbssports.com/nfl/news/most-valuable-nfl-franchises-revealed-cowboys-and-patriots-top-2021-ranking-bills-and-bengals-at-the-bottom/
- https://www.forbes.com/sites/mikeozanian/2021/08/05/the-nfls-most-valuable-teams-2021-average-team-value-soars-to-35-billion-as-league-shrugs-off-pandemic-year/?sh=53fa6e24654e


In [12]:
def get_prices(initPrice= 525000, # https://www.sportico.com/business/sponsorship/2021/sports-sponsorship-media-price-hike-1234645336/ 
                post_value_weight=1, engagement_rate_weight=.75, # Social Media Weights
                audience_weight=1, value_weight=.75, hit_seconds_weight=.5, # Broadcast Weights
                seconds_weight=.5, clarity_weight=.75, size_weight= 1): # Video Board Weights
    """
        Get the prices for the assets
        Input:  None
        Output: Dataframe with prices for assets
    """

    vb_scores = VB_Scoring(seconds_weight, clarity_weight, size_weight)
    bc_scores = BC_Scoring(audience_weight, value_weight, hit_seconds_weight)
    sm_scores = SM_Scoring(post_value_weight, engagement_rate_weight)

    
    bc_prices, initVBPrice = BC_Pricing(bc_scores, initPrice)
    vb_prices, initSMPrice = VB_Pricing(vb_scores, initVBPrice)
    sm_prices = SM_Pricing(sm_scores, initSMPrice)
    overall_prices = overall_Pricing(bc_prices, vb_prices, sm_prices)

    return [overall_prices, bc_prices, vb_prices, sm_prices]

In [13]:
# prints out pricing for each asset category
x = get_prices()

for i in x:
    print(i)
    print()

  Asset Category Bronze Tier Silver Tier Gold Tier
0      Broadcast     182,369     453,458   644,145
1    Video Board     209,451     437,499   520,421
2   Social Media     124,701     303,104   399,918

               Asset Bronze Tier Silver Tier Gold Tier
15        Videoboard     201,667     607,575   933,081
12   SNY Entitlement     329,754     527,405   644,653
0           Ad Tower     114,084     509,199   823,911
5                GIP     118,054     489,516   780,575
6         LED Ribbon     101,967     487,621   792,872
4        Cornerstone     116,853     459,442   721,629
7   LED Tunnel Cover     113,734     446,413   698,690
14    Static Signage     122,359     437,500   672,238
8   LED Tunnel cover     116,614     434,831   672,645
2    CBS Entitlement     275,856     433,718   511,178
9   On-Field Product     122,975     431,688   659,999
10    Press Backdrop     115,636     422,982   649,925
3        CBS Feature     275,100     410,746   465,989
1      CBS Billboard     